# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([67, 73, 32, 10, 49, 33, 20, 14, 81, 25, 25, 25,  4, 32, 10, 10, 69,
       14, 48, 32, 23, 82, 24, 82, 33,  2, 14, 32, 20, 33, 14, 32, 24, 24,
       14, 32, 24, 82, 34, 33, 11, 14, 33, 19, 33, 20, 69, 14, 62, 31, 73,
       32, 10, 10, 69, 14, 48, 32, 23, 82, 24, 69, 14, 82,  2, 14, 62, 31,
       73, 32, 10, 10, 69, 14, 82, 31, 14, 82, 49,  2, 14, 41, 75, 31, 25,
       75, 32, 69, 27, 25, 25, 26, 19, 33, 20, 69, 49, 73, 82, 31])

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

In [6]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from, shape=(1985223,)
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the batch size and number of batches we can make
    batch_size = n_seqs * n_steps # 一个batch的总字符数
    n_batches = len(arr)//batch_size # batch的个数
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size] # 舍弃一些组不成一个batch的字符
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1)) 
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[67 73 32 10 49 33 20 14 81 25]
 [14 32 23 14 31 41 49 14 46 41]
 [19 82 31 27 25 25 28 44 33  2]
 [31 14 72 62 20 82 31 46 14 73]
 [14 82 49 14 82  2 29 14  2 82]
 [14 30 49 14 75 32  2 25 41 31]
 [73 33 31 14 76 41 23 33 14 48]
 [11 14 18 62 49 14 31 41 75 14]
 [49 14 82  2 31 12 49 27 14 47]
 [14  2 32 82 72 14 49 41 14 73]]

y
 [[73 32 10 49 33 20 14 81 25 25]
 [32 23 14 31 41 49 14 46 41 82]
 [82 31 27 25 25 28 44 33  2 29]
 [14 72 62 20 82 31 46 14 73 82]
 [82 49 14 82  2 29 14  2 82 20]
 [30 49 14 75 32  2 25 41 31 24]
 [33 31 14 76 41 23 33 14 48 41]
 [14 18 62 49 14 31 41 75 14  2]
 [14 82  2 31 12 49 27 14 47 73]
 [ 2 32 82 72 14 49 41 14 73 33]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`.

In [9]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. For example,

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow will create different weight matrices for all `cell` objects. Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [10]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
#     lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_size), output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(lstm_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$.

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

In [11]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [13]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

In [14]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


In [15]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

In [16]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 1...  Training loss: 4.4219...  2.7395 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.3347...  2.7063 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.8201...  2.6774 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 4.3343...  2.7309 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 3.9036...  2.6993 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.6604...  2.7510 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.4905...  2.8274 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.3602...  2.7471 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.3321...  2.7229 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.3163...  2.7325 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.2238...  2.8168 sec/batch
Epoch: 1/20...  Training Step: 12...  Training loss: 3.1834...  2.7777 sec/batch
Epoch: 1/20...  Training Step: 13... 

Epoch: 1/20...  Training Step: 103...  Training loss: 3.0789...  2.9427 sec/batch
Epoch: 1/20...  Training Step: 104...  Training loss: 3.0675...  3.0937 sec/batch
Epoch: 1/20...  Training Step: 105...  Training loss: 3.0683...  2.7970 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 3.0699...  2.7977 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 3.0413...  2.8376 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 3.0411...  2.7508 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 3.0556...  2.7960 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 3.0175...  2.8737 sec/batch
Epoch: 1/20...  Training Step: 111...  Training loss: 3.0381...  2.8497 sec/batch
Epoch: 1/20...  Training Step: 112...  Training loss: 3.0446...  2.8300 sec/batch
Epoch: 1/20...  Training Step: 113...  Training loss: 3.0293...  2.7815 sec/batch
Epoch: 1/20...  Training Step: 114...  Training loss: 3.0208...  2.7724 sec/batch
Epoch: 1/20...  

Epoch: 2/20...  Training Step: 203...  Training loss: 2.4799...  2.7285 sec/batch
Epoch: 2/20...  Training Step: 204...  Training loss: 2.4702...  2.8337 sec/batch
Epoch: 2/20...  Training Step: 205...  Training loss: 2.4834...  2.6984 sec/batch
Epoch: 2/20...  Training Step: 206...  Training loss: 2.4717...  2.6403 sec/batch
Epoch: 2/20...  Training Step: 207...  Training loss: 2.4906...  2.6686 sec/batch
Epoch: 2/20...  Training Step: 208...  Training loss: 2.4596...  2.8250 sec/batch
Epoch: 2/20...  Training Step: 209...  Training loss: 2.4519...  2.6523 sec/batch
Epoch: 2/20...  Training Step: 210...  Training loss: 2.4635...  2.6560 sec/batch
Epoch: 2/20...  Training Step: 211...  Training loss: 2.4550...  2.6612 sec/batch
Epoch: 2/20...  Training Step: 212...  Training loss: 2.4855...  2.6789 sec/batch
Epoch: 2/20...  Training Step: 213...  Training loss: 2.4590...  2.8242 sec/batch
Epoch: 2/20...  Training Step: 214...  Training loss: 2.4529...  2.7242 sec/batch
Epoch: 2/20...  

Epoch: 2/20...  Training Step: 303...  Training loss: 2.2269...  2.6676 sec/batch
Epoch: 2/20...  Training Step: 304...  Training loss: 2.2412...  2.7247 sec/batch
Epoch: 2/20...  Training Step: 305...  Training loss: 2.2219...  2.7628 sec/batch
Epoch: 2/20...  Training Step: 306...  Training loss: 2.2656...  2.6902 sec/batch
Epoch: 2/20...  Training Step: 307...  Training loss: 2.2462...  2.8842 sec/batch
Epoch: 2/20...  Training Step: 308...  Training loss: 2.2120...  2.9796 sec/batch
Epoch: 2/20...  Training Step: 309...  Training loss: 2.2346...  2.9183 sec/batch
Epoch: 2/20...  Training Step: 310...  Training loss: 2.2423...  2.8706 sec/batch
Epoch: 2/20...  Training Step: 311...  Training loss: 2.2206...  2.7463 sec/batch
Epoch: 2/20...  Training Step: 312...  Training loss: 2.2158...  2.7950 sec/batch
Epoch: 2/20...  Training Step: 313...  Training loss: 2.2089...  2.7664 sec/batch
Epoch: 2/20...  Training Step: 314...  Training loss: 2.1817...  2.8838 sec/batch
Epoch: 2/20...  

Epoch: 3/20...  Training Step: 403...  Training loss: 2.0671...  3.0814 sec/batch
Epoch: 3/20...  Training Step: 404...  Training loss: 2.0680...  2.8037 sec/batch
Epoch: 3/20...  Training Step: 405...  Training loss: 2.0894...  3.2647 sec/batch
Epoch: 3/20...  Training Step: 406...  Training loss: 2.0559...  3.1375 sec/batch
Epoch: 3/20...  Training Step: 407...  Training loss: 2.0409...  3.1918 sec/batch
Epoch: 3/20...  Training Step: 408...  Training loss: 2.0367...  3.1091 sec/batch
Epoch: 3/20...  Training Step: 409...  Training loss: 2.0561...  3.0804 sec/batch
Epoch: 3/20...  Training Step: 410...  Training loss: 2.0925...  3.2133 sec/batch
Epoch: 3/20...  Training Step: 411...  Training loss: 2.0590...  3.1714 sec/batch
Epoch: 3/20...  Training Step: 412...  Training loss: 2.0320...  3.2137 sec/batch
Epoch: 3/20...  Training Step: 413...  Training loss: 2.0564...  3.1611 sec/batch
Epoch: 3/20...  Training Step: 414...  Training loss: 2.0919...  3.1918 sec/batch
Epoch: 3/20...  

...

Epoch: 18/20...  Training Step: 3559...  Training loss: 1.0813...  2.5993 sec/batch
Epoch: 18/20...  Training Step: 3560...  Training loss: 1.0681...  2.6789 sec/batch
Epoch: 18/20...  Training Step: 3561...  Training loss: 1.0539...  2.5997 sec/batch
Epoch: 18/20...  Training Step: 3562...  Training loss: 1.0792...  2.6377 sec/batch
Epoch: 18/20...  Training Step: 3563...  Training loss: 1.0742...  2.6433 sec/batch
Epoch: 18/20...  Training Step: 3564...  Training loss: 1.0685...  2.6225 sec/batch
Epoch: 19/20...  Training Step: 3565...  Training loss: 1.2036...  2.5666 sec/batch
Epoch: 19/20...  Training Step: 3566...  Training loss: 1.0956...  2.5289 sec/batch
Epoch: 19/20...  Training Step: 3567...  Training loss: 1.0754...  2.5925 sec/batch
Epoch: 19/20...  Training Step: 3568...  Training loss: 1.1101...  2.6342 sec/batch
Epoch: 19/20...  Training Step: 3569...  Training loss: 1.0734...  2.6420 sec/batch
Epoch: 19/20...  Training Step: 3570...  Training loss: 1.0564...  2.6008 se

Epoch: 19/20...  Training Step: 3657...  Training loss: 1.0502...  2.6322 sec/batch
Epoch: 19/20...  Training Step: 3658...  Training loss: 1.0409...  2.5787 sec/batch
Epoch: 19/20...  Training Step: 3659...  Training loss: 1.0631...  2.5741 sec/batch
Epoch: 19/20...  Training Step: 3660...  Training loss: 1.0832...  2.6059 sec/batch
Epoch: 19/20...  Training Step: 3661...  Training loss: 1.0759...  2.6071 sec/batch
Epoch: 19/20...  Training Step: 3662...  Training loss: 1.0357...  2.6391 sec/batch
Epoch: 19/20...  Training Step: 3663...  Training loss: 1.0584...  2.5528 sec/batch
Epoch: 19/20...  Training Step: 3664...  Training loss: 1.0513...  2.5757 sec/batch
Epoch: 19/20...  Training Step: 3665...  Training loss: 1.0623...  2.6333 sec/batch
Epoch: 19/20...  Training Step: 3666...  Training loss: 1.0618...  2.6809 sec/batch
Epoch: 19/20...  Training Step: 3667...  Training loss: 1.0828...  2.6091 sec/batch
Epoch: 19/20...  Training Step: 3668...  Training loss: 1.0504...  2.5777 se

Epoch: 19/20...  Training Step: 3755...  Training loss: 1.0435...  2.6079 sec/batch
Epoch: 19/20...  Training Step: 3756...  Training loss: 1.0470...  2.5751 sec/batch
Epoch: 19/20...  Training Step: 3757...  Training loss: 1.0664...  2.6908 sec/batch
Epoch: 19/20...  Training Step: 3758...  Training loss: 1.0556...  2.6368 sec/batch
Epoch: 19/20...  Training Step: 3759...  Training loss: 1.0431...  2.6803 sec/batch
Epoch: 19/20...  Training Step: 3760...  Training loss: 1.0636...  2.5919 sec/batch
Epoch: 19/20...  Training Step: 3761...  Training loss: 1.0572...  2.5893 sec/batch
Epoch: 19/20...  Training Step: 3762...  Training loss: 1.0550...  2.5776 sec/batch
Epoch: 20/20...  Training Step: 3763...  Training loss: 1.1844...  2.6216 sec/batch
Epoch: 20/20...  Training Step: 3764...  Training loss: 1.0801...  2.6548 sec/batch
Epoch: 20/20...  Training Step: 3765...  Training loss: 1.0554...  2.6003 sec/batch
Epoch: 20/20...  Training Step: 3766...  Training loss: 1.0927...  2.5777 se

Epoch: 20/20...  Training Step: 3853...  Training loss: 1.0409...  2.6138 sec/batch
Epoch: 20/20...  Training Step: 3854...  Training loss: 1.0523...  2.5763 sec/batch
Epoch: 20/20...  Training Step: 3855...  Training loss: 1.0372...  2.5729 sec/batch
Epoch: 20/20...  Training Step: 3856...  Training loss: 1.0269...  2.5947 sec/batch
Epoch: 20/20...  Training Step: 3857...  Training loss: 1.0484...  2.6225 sec/batch
Epoch: 20/20...  Training Step: 3858...  Training loss: 1.0688...  2.5630 sec/batch
Epoch: 20/20...  Training Step: 3859...  Training loss: 1.0609...  2.5872 sec/batch
Epoch: 20/20...  Training Step: 3860...  Training loss: 1.0240...  2.6223 sec/batch
Epoch: 20/20...  Training Step: 3861...  Training loss: 1.0446...  2.6821 sec/batch
Epoch: 20/20...  Training Step: 3862...  Training loss: 1.0388...  2.5696 sec/batch
Epoch: 20/20...  Training Step: 3863...  Training loss: 1.0471...  2.5909 sec/batch
Epoch: 20/20...  Training Step: 3864...  Training loss: 1.0479...  2.5704 se

Epoch: 20/20...  Training Step: 3951...  Training loss: 1.0423...  2.5772 sec/batch
Epoch: 20/20...  Training Step: 3952...  Training loss: 1.0606...  2.5905 sec/batch
Epoch: 20/20...  Training Step: 3953...  Training loss: 1.0346...  2.6036 sec/batch
Epoch: 20/20...  Training Step: 3954...  Training loss: 1.0432...  2.6674 sec/batch
Epoch: 20/20...  Training Step: 3955...  Training loss: 1.0548...  2.6038 sec/batch
Epoch: 20/20...  Training Step: 3956...  Training loss: 1.0414...  2.5892 sec/batch
Epoch: 20/20...  Training Step: 3957...  Training loss: 1.0342...  2.5783 sec/batch
Epoch: 20/20...  Training Step: 3958...  Training loss: 1.0558...  2.6327 sec/batch
Epoch: 20/20...  Training Step: 3959...  Training loss: 1.0478...  2.6311 sec/batch
Epoch: 20/20...  Training Step: 3960...  Training loss: 1.0426...  2.5564 sec/batch


#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [17]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i3960_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3200_l512.ckpt"
all_mo

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [18]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [19]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [20]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i3960_l512.ckpt'

In [21]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

Far5"3X8**1111****11143* *3. Dolly it an effer
we ded three young letters we'll take that suncer."

The colonel interpepted him. "What a good there is a thich dear, and
I'm satisfied with a sudden in the studen. Alexey Alexandrovitch," she
added, lay, and also as she han excheed far from simply for
her, and felt to talk in the drawing room, and went out.

"Yes, I shall see that!"

And all sildey stopped, was futtle and helpless scruminy, and he stopped,
she was terrible, and he was so far from the door, and so sheally careed
for her son stumble that he was not enjoying it to see them in the
sacromman with her son, who could not be sent to all her face;
business of this thoughts at any other in the carriage, that had
sent to him the general work of her, then it will happen of the sound,
though it all of to the serfate sudden, and he went into the doctor, which he
would fing that infered the most intense, thoughts were te get a ballstake,
and he started still, looked at him, stood still 

In [22]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farnthea t oh tit shan tins. ""
""
"he s titise he hing so her,

fhe thed thit her, sang the the sase wiress an as he hos to this har at her we hes thas and wing wirthe sins sed shes hon hind as he her ande nere whe ton sh se shan te he shere sost ond this shes sitis thisg wit sante to the se hing ans on ser sos on ass ate the hind tot an hos ang she te was he te wan thar and hh wes ans hhe hit thit he the se tit or sor to see and sond at he won hins at atis hos sont al sos ange the hes hasd torin so sos so sind sir ald het hind sis on tore sing hh t on as on hid se hes ang so thim se and hon an tere wout shit ha the be wha gishe tha sestes ang th ris se th at,e
 hor th shit so was tha sos or he wand sothin thes ale wos thas this at tar the winge se thind wot an an hote he wor and to he wit she has he th the sars ont on he thim th she be than te whon the her and he ser ord to hed her he hin ase, wete bisthe th se ta tho thin wins satin the son hind an shores tos shed til wis ant al wor

In [23]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farzirkn at were of in the moster and
his the shours for hid hers on her his crintered oner in her
treat that he chatre sanco the share, hearing the meath her and
to hars of
herss carrse and
crupse to the roon to her and have sead,"
"And he's sander of inco a man the mone were sanding as at wish thines of a mustines ond wand ther werl and asente of she hersall and suppos on the routher was her, sanced a derand to all and that to the serones then ane alowe ous
astion," he said has brencersed at in all that it, and the
painted to she
whates this chinched."


She we haden sto day to the rackine he stile some hersted him her suppested thing han supees of
ticully. He rown then him healle.


Alxay Alkadyovitch had becimed andirsess ond andoul tho groom," said Anex Alaxaderovitch sailing at the
prater had seeping that he was tood and anster to the
promer ound
the comenters, was no sould, and the caster ane the monters, she southes who can shinest who have on the sard the stolles of his
cament

In [24]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farjuccy it, the most at the same that where her briegh, with all shere sont tors that they add to to be him
fach, simplated the senvite, and taken too the same
to his saming and how teere a greating and his face of
hearth, but she there hasted his been for him to to the peacan sinking of
his sawand, and howed, and his face was no look, the can through stend her hand, and wanted himself--he would have a persoover he did not sake her tarking to him, to the stall of the hissele, that was
the some to this positems and talking at the
siddle, and he heard him had been speaking at the streng of she sead he had but her sood. The pirter, whither he had beed
that his seak he cromd
an her stard a smetther strangs of standing
by the pack that
she was to be brother at has been surdenly through finss into the pring and this heart fill have a road
thinked to have been a stare aftard and both and betteen in a sterry.

"Yasher a mosther instarrat of him as she cauged at
a she with a charrian of her. H